In [2]:
import tensorflow as tf
import numpy as np
from keras.layers import Input,Lambda,Conv2D, MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras.models import load_model, Model
import keras.backend as K
import random
import colorsys
import imghdr
import functools
from functools import partial, reduce
from keras.layers.merge import concatenate

'''
darknet19
'''
def compose(*funcs):
    if funcs:
        return reduce(lambda f,g:lambda *a,**kw:g(f(*a,**kw)), funcs)
    else:
        raise ValueError('composition of empty sequence not supported')
        
_DarknetConv2D = partial(Conv2D, padding='same')

@functools.wraps(Conv2D)
def DarknetConv2D(*args, **kwargs):
    darknet_conv_kwargs = {'kernel_regularizer': l2(5e-4)}
    darknet_conv_kwargs.update(kwargs)
    return _DarknetConv2D(*args, **darknet_conv_kwargs)

def DarknetConv2D_BN_Leaky(*args, **kwargs):
    no_bias_kwargs = {'use_bias':False}
    no_bias_kwargs.update(kwargs)
    return compose(DarknetConv2D(*args,**no_bias_kwargs),BatchNormalization(),LeakyReLU(alpha=0.1))

def bottleneck_block(outer_filters, bottleneck_filters):
    return compose(DarknetConv2D_BN_Leaky(outer_filters, (3,3)),
                   DarknetConv2D_BN_Leaky(bottleneck_filters, (1,1)),
                   DarknetConv2D_BN_Leaky(outer_filters, (3,3)))

def bottleneck_x2_block(outer_filters, bottleneck_filters):
    return compose(bottleneck_block(outer_filters, bottleneck_filters),
                  DarknetConv2D_BN_Leaky(bottleneck_filters, (1,1)),
                  DarknetConv2D_BN_Leaky(outer_filters, (3,3)))

def darknet_body():
    return compose(DarknetConv2D_BN_Leaky(32,(3,3)),
                  MaxPooling2D(),
                  DarknetConv2D_BN_Leaky(64,(3,3)),
                  MaxPooling2D(),
                  bottleneck_block(128,64),
                  MaxPooling2D(),
                  bottleneck_block(256, 128),
                  MaxPooling2D(),
                  bottleneck_x2_block(512, 256),
                  MaxPooling2D(),
                  bottleneck_x2_block(1024, 512))

def darknet19(inputs):
    body = darknet_body()(inputs)
    logits = DarknetConv2D(1000,(1,1),activation='softmax')(body)
    return Model(inputs, logits)

'''
keras yolo
'''
voc_anchors = np.array([[1.08,1.19],[3.42,4.41],[6.63,11.38],[9.42,5.11],[16.62,10.52]])
voc_classes = [
    "aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat",
    "chair", "cow", "diningtable", "dog", "horse", "motorbike", "person",
    "pottedplant", "sheep", "sofa", "train", "tvmonitor"
]

def space_to_depth_x2(x):
    return tf.space_to_depth(x, block_size=2)

def space_to_depth_x2_output_shape(input_shape):
    if input_shape[1]:
        return (input_shape[0],input_shape[1]//2,input_shape[2]//2,4*input_shape[3])
    else:
        return (input_shape[0],None, None, 4*input_shape[3])
    
def yolo_body(inputs, num_anchors, num_classes):
    darknet = Model(inputs, darknet_body()(inputs))
    conv20 = compose(DarknetConv2D_BN_Leaky(1024,(3,3)),
                    DarknetConv2D_BN_Leaky(1024, (3,3)))(darknet.output)
    conv13 = darknet.layers[43].output
    conv21 = DarknetConv2D_BN_Leaky(64,(1,1))(conv13)
    conv21_reshaped = Lambda(space_to_depth_x2,
                            output_shape = space_to_depth_x2_output_shape,
                            name = 'space_to_depth')(conv21)
    
    x = concatenate([conv21_reshaped, conv20])
    x = DarknetConv2D_BN_Leaky(1024,(3,3))(x)
    x = DarknetConv2D(num_anchors * (num_classes+5), (1,1))(x)
    return Model(inputs, x)

def yolo_head(feats, anchors, num_classes):
    num_anchors = len(anchors)
    anchors_tensor = K.reshape(K.variable(anchors), [1,1,1,num_anchors,2])
    
    conv_dims = K.shape(feats)[1:3]
    conv_height_index = K.arange(0, stop=conv_dims[0])
    conv_width_index = K.arange(0, stop=conv_dims[1])
    conv_height_index = K.tile(conv_height_index, [conv_dims[1]])
    
    conv_width_index = K.tile(K.expand_dims(conv_width_index,0), [conv_dims[0], 1])
    conv_width_index = K.flatten(K.transpose(conv_width_index))
    conv_index = K.transpose(K.stack([conv_height_index, conv_width_index]))
    conv_index = K.reshape(conv_indexx, [1, conv_dims[0], conv_dims[1], 1, 2])
    conv_index = K.cast(conv_index, K.dtype(feats))
    
    feats = K.reshape(feats, [-1, conv_dims[0], conv_dims[1], num_anchors, num_classes + 5])
    conv_dims = K.cast(K.reshape(conv_dims, [1,1,1,1,2]), K.dtype(feats))
    
    box_confidence = K.sigmoid(feats[..., 4:5])
    box_xy = K.sigmoid(feats[...,:2])
    box_wh = K.exp(feats[...,2:4])
    box_class_probs = K.softmax(feats[...,5:])
    return box_confidence, box_xy, box_wh, box_class_probs

def yolo_loss(args, anchors, num_classes, rescore_confidence=False, print_loss=False):
    (yolo_output,true_boxes,detectors_mask,matching_true_boxes) = args
    num_anchors = len(anchors)
    object_scale = 5
    no_object_scale = 1
    class_scale = 1
    coordinates_scale = 1
    pred_xy, pred_wh, pred_confidence, pred_class_prob = yolo_head(yolo_output, anchors, num_classes)
    
    yolo_output_shape = K.shape(yolo_output)
    feats = K.reshape(yolo_output,[-1,yolo_output_shape[1], yolo_output_shape[2], num_anchors, num_classes+5])
    pred_boxes = K.concatenate((K.sigmoid(feats[...,0:2]),feats[...,2:4]), axis=-1)
    
    pred_xy = K.expend_dims(pred_xy, 4)
    pred_wh = K.expand_dims(pred_wh, 4)
    
    pred_wh_half = pred_wh/2.
    pred_mins = pred_xy - pred_wh_half
    pred_maxes = pred_xy + pred_wh_half
    
    true_boxes_shape = K.shape(true_boxes)
    
    true_boxes = K.reshape(true_boxes, [true_boxes_shape[0],1,1,1,true_boxes_shape[1], true_boxes_shape[2]])
    true_xy = true_boxes[...,0:2]
    true_wh = true_boxes[...,2:4]
    
    true_wh_half = true_wh / 2.
    true_mins = true_xy - true_wh_half
    true_maxes = true_xy + true_wh_half
    
    intersect_mins = k.maximum(pred_mins, true_mins)
    intersect_maxes = K.minimum(pred_maxes, true_maxes)
    intersect_wh = K.maximum(intersect_maxes - intersect_mins, 0.)
    intersect_area = intersect_wh[...,0] * intersect_wh[...,1]
    
    pred_area = pred_wh[...,0] * pred_wh[...,1]
    true_area = true_wh[...,0] * true_wh[...,1]
    
    union_area = pred_areas + true_area - intersect_area
    iou_score = intersect_area/union_area
    
    best_ious = K.max(iou_score, axis=4)
    best_ious = K.expend_dims(best_ious)
    
    object_detections = K.castst(best_iou>0.6, K.dtype(best_ious))
    
    no_object_weights = (no_object_scale*(1-object_detections) * (1-detectors_mask))
    no_object_loss = no_object_weights * K.square(-pred_confidence)
    
    if rescore_confidence:
        object_loss = (object_scale * detectors_mask * K.square(best_ious-pred_confidence))
    else:
        object_loss = (object_scale * detectors_mask * K.square(1-pred_confidence))
    confidence_loss = objects_loss + no_object_loss
    
    matching_classes = K.castst(matching_true_boxes[...,4], 'int32')
    matching_classes = K.one_hot(matching_classes, num_classes)
    classification_loss = (class_scale*detectors_mask*K.square(matching_classes-pred_class_prob))
    
    matching_boxes = matching_true_boxes[...,0:4]
    coordinates_loss = (coordinates_scale*detectors_mask*K.square(matching_boxes-pred_boxes))
    
    confidence_loss_sum = K.sum(confidence_loss)
    classification_loss_sum = K.sum(classification_loss)
    coordinate_loss_sum = K.sum(coordinates_loss)
    total_loss = 0.5 * (confidence_loss_sum+classification_loss_sum+coordinate_loss_sum)
    if print_loss:
        total_loss = tf.Print(total_loss,[total_loss,confidence_loss_sum,classification_loss_sum,
                                         coordinate_loss_sumd],
                              message='yolo_loss, conf_loss,class_loss,box_coord_loss:')
    return total_loss

def yolo(inputs, anchors, num_classes):
    num_anchors = len(anchors)
    body = yolo_body(inputs, num_anchors, num_classes)
    outputs = yolo_head(body.output, anchors, num_classes)
    return outputs

def preprocess_true_boxes(true_boxes, anchors, image_size):
    height, width = image_size
    num_anchors = len(anchors)
    
#     assert height%32 == 0, 'image sizes in yolo_v2 must be multiples of 32.'
#     assert width%32 == 0,
    conv_height = height // 32
    conv_width = width // 32
    num_box_params = true_boxes.shape[1]
    detectors_mask = np.zeros((conv_height, conv_width, num_anchors, 1), dtype=np.float32)
    matching_true_boxes = np.zeros((conv_height, conv_width, num_anchors, num_box_params), dtype=np.float32)
    
    for box in true_boxes:
        box_class = box[4:5]
        box = box[0:4]*np.array([conv_width, conv_height, conv_width, conv_height])
        i = np.floor(box[1]).astype('int')
        j = min(np.floor(box[0]).astype('int'), 1)
        best_iou = 0
        best_anchor = 0
        
        for k, anchor in enumerate(anchors):
            box_maxes = box[2:4]/2.
            box_mins = -box_maxes
            anchor_maxes = (anchor/2.)
            anchor_mins = -anchor_maxes
            
            intersect_mins = np.maximum(box_mins, anchor_mins)
            intersect_maxes = np.minimum(box_maxes, anchors_maxes)
            intersect_wh = np.maximum(intersect_maxesxes - intersect_mins, 0.)
            intersect_area = intersect_wh[0] * intersect+wh[1]
            box_area = box[2] * box[3]
            anchor_area = anchor[0] * anchor[1]
            iou = intersect_area / (box_area+anchor_area-intersect_area)
            if iou > best_iou:
                best_iou = iou
                best_anchorr = k
        
        if best_iou > 0:
            detectors_mask[i,j,best_anchor] = 1
            adjusted_box = np.array([
                box[0] - j, box[1] - i,
                np.log(box[2]/anchors[best_anchor][0]),
                np.log(box[3]/anchors[best_anchor][1]),
                box_class
            ], dtype = np.float32)
            matching_true_boxes[i,j,best_anchor] = adjusted_box
    return detectors_mask, matching_true_boxes

Using TensorFlow backend.


NameError: name 'np' is not defined